In [ ]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
from phoenix.server.api.types.pagination import (
    NodeIdentifier,
    SortableField,
    SortableFieldType,
)

project_id = "UHJvamVjdDox"
client = Client(
    transport=RequestsHTTPTransport(url="http://127.0.0.1:6006/graphql", timeout=100),
    fetch_schema_from_transport=True,
)

In [ ]:
cursor = "MTAwOkRBVEVUSU1FOjIwMjMtMTItMTFUMTc6NDQ6MDIuNTM0MTI5KzAwOjAw"
node_identifier = NodeIdentifier.from_cursor(cursor)
print(node_identifier)

In [ ]:
# test query for doing sanity checks
response = client.execute(
    gql(
        """query SpansTableSpansQuery($after: String = null, $filterCondition: String = null, $first: Int = 100, $sort: SpanSort = {col: startTime, dir: desc}, $timeRange: TimeRange, $id: GlobalID!) {
  node(id: $id) {
    ... on Project {
      spans(
        first: $first
        after: $after
        sort: $sort
        filterCondition: $filterCondition
        timeRange: $timeRange
      ) {
        edges {
          cursor
        }
        pageInfo {
          endCursor
          hasNextPage
        }
      }
    }
  }
}"""
    ),
    variable_values={
        "after": None,
        "filterCondition": "",
        "first": 100,
        "sort": {"col": "startTime", "dir": "desc"},
        "timeRange": {
            "start": "2024-04-28T17:00:00.000Z",
            "end": "2025-05-05T17:00:00.000Z",
        },
        "id": "UHJvamVjdDox",
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
new_ids

In [ ]:
spans_query = gql(
    """query SpansQuery($projectId: GlobalID!, $after: String = null, $before: String = null, $filterCondition: String = null, $first: Int = null, $last: Int = null, $sort: SpanSort = null) {
  node(id: $projectId) {
    ... on Project {
      spans(
        after: $after
        before: $before
        filterCondition: $filterCondition
        first: $first
        last: $last
        rootSpansOnly: false
        sort: $sort
      ) {
        edges {
          cursor
        }
        pageInfo {
          hasNextPage
          hasPreviousPage
          startCursor
          endCursor
        }
      }
    }
  }
}"""
)

In [ ]:
# basic query
response = client.execute(
    spans_query,
    variable_values={"projectId": project_id, "first": 5},
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [765, 764, 763, 762, 761], new_ids

In [ ]:
# query with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(rowid=761).to_cursor(),
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [760, 759, 758, 757, 756], new_ids

In [ ]:
# page ends on the penultimate record and excludes last record
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(7).to_cursor(),
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert new_ids == [6, 5, 4, 3, 2], new_ids
assert has_next_page is True
assert has_previous_page is False

In [ ]:
# page ends on the last record exactly
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(6).to_cursor(),
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert new_ids == [5, 4, 3, 2, 1], new_ids
assert has_next_page is False
assert has_previous_page is False

In [ ]:
# page ends before it reaches the limit
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(5).to_cursor(),
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert new_ids == [4, 3, 2, 1], new_ids
assert has_next_page is False
assert has_previous_page is False

In [ ]:
# basic filter condition
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "first": 5,
        "filterCondition": "span_kind == 'LLM'",
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [
    765,
    760,
    755,
    750,
    745,
], new_ids

In [ ]:
# basic filter condition with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "first": 5,
        "after": NodeIdentifier(
            765
        ).to_cursor(),  # skip the first span satisfying the filter condition
        "filterCondition": "span_kind == 'LLM'",
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [
    760,
    755,
    750,
    745,
    740,
], new_ids

In [ ]:
# compound filter condition with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(
            745
        ).to_cursor(),  # skip the first span satisfying the filter condition
        "first": 5,
        "filterCondition": "span_kind == 'LLM' and cumulative_llm_token_count_prompt > 300",
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [
    740,
    730,
    720,
    710,
    690,
], new_ids

In [ ]:
# order by descending start time
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "desc"},
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
end_cursor = response["node"]["spans"]["pageInfo"]["endCursor"]
end_node_identifier = NodeIdentifier.from_cursor(end_cursor)
assert new_ids == [765, 764, 763, 762, 761], new_ids
assert end_node_identifier.rowid == 761
assert (end_sortable_field := end_node_identifier.sortable_field) is not None
assert (
    end_node_start_timestamp := end_sortable_field.value.isoformat()
) == "2023-12-11T17:48:40.807667+00:00", end_node_start_timestamp

In [ ]:
# order by ascending start time
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "asc"},
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
end_cursor = response["node"]["spans"]["pageInfo"]["endCursor"]
end_node_identifier = NodeIdentifier.from_cursor(end_cursor)
assert new_ids == [1, 2, 3, 4, 5], new_ids
assert end_node_identifier.rowid == 5
assert (end_sortable_field := end_node_identifier.sortable_field) is not None
assert (
    end_node_start_timestamp := end_sortable_field.value.isoformat()
) == "2023-12-11T17:43:23.712144+00:00", end_node_start_timestamp

In [ ]:
# order by cumulative prompt token count in descending order
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "cumulativeTokenCountPrompt", "dir": "desc"},
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [
    710,
    709,
    706,
    645,
    644,
], new_ids

In [ ]:
# order by cumulative prompt token count in ascending order
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "cumulativeTokenCountPrompt", "dir": "asc"},
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [763, 762, 758, 757, 753], new_ids

In [ ]:
# order by descending start time with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "desc"},
        "first": 5,
        "after": NodeIdentifier(
            760,
            sortable_field=SortableField.from_stringified_value(
                type=SortableFieldType.DATETIME,
                stringified_value="2023-12-11T17:48:40.154938+00:00",
            ),
        ).to_cursor(),
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
start_cursor = response["node"]["spans"]["pageInfo"]["startCursor"]
end_cursor = response["node"]["spans"]["pageInfo"]["endCursor"]
start_node_identifier = NodeIdentifier.from_cursor(start_cursor)
end_node_identifier = NodeIdentifier.from_cursor(end_cursor)
assert new_ids == [759, 758, 757, 756, 755], new_ids
assert (start_sortable_field := start_node_identifier.sortable_field) is not None
assert (
    start_timestamp := start_sortable_field.stringify_value()
) == "2023-12-11T17:48:40.154139+00:00", start_timestamp
assert (
    start_field_type := start_sortable_field.type
) == SortableFieldType.DATETIME, start_field_type
assert (end_sortable_field := end_node_identifier.sortable_field) is not None
assert (
    end_timestamp := end_sortable_field.stringify_value()
) == "2023-12-11T17:48:38.803725+00:00", end_timestamp
assert (end_field_type := end_sortable_field.type) == SortableFieldType.DATETIME, end_field_type

In [ ]:
# order by ascending start time with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "asc"},
        "first": 5,
        "after": NodeIdentifier(
            8,
            sortable_field=SortableField.from_stringified_value(
                type=SortableFieldType.DATETIME,
                stringified_value="2023-12-11T17:43:25.540677+00:00",
            ),
        ).to_cursor(),
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
start_cursor = response["node"]["spans"]["pageInfo"]["startCursor"]
end_cursor = response["node"]["spans"]["pageInfo"]["endCursor"]
start_node_identifier = NodeIdentifier.from_cursor(start_cursor)
end_node_identifier = NodeIdentifier.from_cursor(end_cursor)
assert new_ids == [9, 10, 11, 12, 13], new_ids
assert (start_sortable_field := start_node_identifier.sortable_field) is not None
assert (
    start_timestamp := start_sortable_field.stringify_value()
) == "2023-12-11T17:43:25.842986+00:00", start_timestamp
assert (
    start_field_type := start_sortable_field.type
) == SortableFieldType.DATETIME, start_field_type
assert (end_sortable_field := end_node_identifier.sortable_field) is not None
assert (
    end_timestamp := end_sortable_field.stringify_value()
) == "2023-12-11T17:43:26.496177+00:00", end_timestamp
assert (end_field_type := end_sortable_field.type) == SortableFieldType.DATETIME, end_field_type